In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

df = pd.read_csv('../input/reviews/5556-ar-reviews.csv')
df = df.sample(frac=1)
df.head()

In [ ]:

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [ ]:


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [word for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1

    return bag


stopwords = {'فإذا', 'أنى', 'بمن', 'حتى', 'لم', 'أنتما', 'هناك', 'تينك', 'بل', 'إي', 'عن', 'ولكن', 'وإذا', 'دون', 'إنا', 'إذن', 'بكم', 'حين', 'عند', 'هل', 'إلا', 'هاته', 'ذينك', 'اللواتي', 'كذا', 'لستما', 'هي', 'اللتان', 'أكثر', 'كلتا', 'لكن', 'ليستا', 'هكذا', 'عسى', 'إذ', 'إن', 'اللاتي', 'إذا', 'بهم', 'نحن', 'فيما', 'ذاك', 'بكن', 'بيد', 'لهن', 'هذي', 'كأي', 'ذوا', 'أي', 'كلاهما', 'هذين', 'أينما', 'كي', 'إليكن', 'ماذا', 'هيا', 'هنالك', 'بي', 'بما', 'تلكما', 'بعض', 'بهن', 'تين', 'ريث', 'على', 'غير', 'حيثما', 'كأن', 'بخ', 'هاتان', 'هاهنا', 'ما', 'هيهات', 'لدى', 'شتان', 'لسنا', 'كيفما', 'مع', 'ممن', 'كما', 'إنما', 'يا', 'عليه', 'لك', 'ذه', 'ذان', 'لهما', 'ليست', 'لنا', 'مه', 'أنتن', 'في', 'لولا', 'بس', 'لها', 'أقل', 'عليك', 'فلا', 'مهما', 'ليسا', 'ذين', 'ذات', 'كلما', 'ذا', 'ذو', 'فيه', 'تي', 'هنا', 'هاتين', 'ها', 'هم', 'ألا', 'لا', 'سوى', 'وإذ', 'كم', 'لست', 'حيث', 'إليكما', 'لوما', 'الذين', 'كلا',
             'التي', 'كأين', 'ذواتي', 'لستم', 'هذا', 'فمن', 'ذلكم', 'وما', 'كيف', 'لكم', 'حاشا', 'بك', 'والذي', 'أن', 'لهم', 'لسن', 'ثمة', 'ذي', 'وإن', 'ومن', 'أيها', 'له', 'متى', 'بلى', 'اللتين', 'لستن', 'بكما', 'قد', 'كليكما', 'لكما', 'هلا', 'آي', 'لكنما', 'اللذين', 'اللائي', 'ذلكن', 'لاسيما', 'ذلك', 'مذ', 'اللتيا', 'هما', 'إليك', 'سوف', 'منها', 'والذين', 'أنتم', 'هاتي', 'لكي', 'اللذان', 'ذواتا', 'عما', 'فيها', 'إلى', 'تلك', 'كل', 'لي', 'هو', 'فيم', 'إليكم', 'بها', 'ذانك', 'إنه', 'هؤلاء', 'أولئك', 'إذما', 'بنا', 'من', 'خلا', 'ليسوا', 'ثم', 'لعل', 'وهو', 'نحو', 'أين', 'لئن', 'عدا', 'آه', 'كأنما', 'كليهما', 'الذي', 'لن', 'نعم', 'هذه', 'بهما', 'ليت', 'تلكم', 'أما', 'منذ', 'أو', 'هاك', 'بماذا', 'كذلك', 'أنا', 'آها', 'فإن', 'عل', 'منه', 'هيت', 'أف', 'أم', 'إيه', 'كيت', 'ته', 'لكيلا', 'ليس', 'مما', 'هذان', 'أنت', 'حبذا', 'ولو', 'أوه', 'إما', 'لو', 'بين', 'به', 'ولا', 'لما', 'بعد', 'هن', 'ذلكما', 'أولاء', 'و'}


def tokenize(sentence):
    tmpTokens = sentence.lower().split()
    tokens = [token for token in tmpTokens if (
        (token not in stopwords) and (len(token) > 0))]

    return tokens

In [ ]:


all_words = []
tags = [0, 1]
xy = []
# loop through each sentence in our intents patterns
intents = df[["label", "text"]].values

for pattern in intents:
    tag = pattern[0]
    # tokenize each word in the sentence
    w = tokenize(pattern[1])
    # add to our words list
    all_words.extend(w)
    # add to xy pair
    xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [w for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters
num_epochs = 100
batch_size = 64
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)  # 59915 2



In [ ]:

class ReviewsDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples


dataset = ReviewsDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for idx, (words, labels) in enumerate(train_loader):
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()/idx:.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data2.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')


# Prediction

In [ ]:
sentence= "رحلاتي القادمة الى الرياض عندهم أن شاء الله . . طاقم العمل رائع كلهم ا.وائل و اثنين شباب في فترة الصباح ناسي اسمهم . تقديم ضيافة عند القدوم صباحا . تقديم ضيافة في المساء شاي و قهوة عربي مع تمر . راحة و نظافة المكان جدا جدا نظيف و خدمة الغرف سريعة . موقعه مناسب للأشخاص اهتمامهم السفارات او الجامعة قريب منها و لكن بعيد عن شرق الرياض . سهولة الوصول الى الدائري الشمالي . هدوء المكان . جميع الخدمات حولك . المسجد مقابل الفندق. لا يوجد شي سي ابدا" 

sentence = tokenize(sentence)
X = bag_of_words(sentence, all_words)
X = X.reshape(1, X.shape[0])
X = torch.from_numpy(X).to(device)

output = model(X)
_, predicted = torch.max(output, dim=1)

tag = tags[predicted.item()]

probs = torch.softmax(output, dim=1)
prob = probs[0][predicted.item()]
if prob.item() > 0.75:
    for intent in intents:
        if tag == intent[0]:
            print(f" {tag} ")
            break
else:
    print("I do not understand...")
